<a href="https://colab.research.google.com/github/BiakkoNoMiko/homework_projects/blob/main/Autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Text preparation for NLP

In [34]:
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
from tqdm import tqdm
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download("stopwords")
nltk.download('punkt')

In [29]:
# preproc text to get words only

en_ru_words = re.compile("[a-zа-яё]+")

def words_only(text):
    return " ".join(en_ru_words.findall(text.lower()))

In [30]:
# read file
df = pd.read_excel(/content/Volchkevich_8_klass_Geometria.xlsx)

In [36]:
#preproc text

df['task'] = df['task'].progress_apply(words_only)

my bar!: 100%|██████████| 853/853 [00:00<00:00, 49218.49it/s]


In [37]:
df.head()

,task,answer,solution,paint,topic,subtopic,кол-во звезд
0,на рисунке снизу показана так называемая штурм...,NaN,NaN,2,Параллелограмм,NaN,NaN
1,на рисунке показан шарнирный механизм который ...,NaN,NaN,3,Параллелограмм,NaN,NaN
2,на рисунке справа показаны два колеса паровоза...,NaN,NaN,4,Параллелограмм,NaN,NaN
3,на рисунке слева показано устройство шарнирног...,NaN,NaN,5,Параллелограмм,NaN,NaN
4,нарисуйте две окружности одинакового радиуса с...,NaN,NaN,6,Параллелограмм,Признак,NaN


In [216]:
%%time
vec = TfidfVectorizer(stop_words=stopwords.words('english'),
                     #   max_features=5000,
                        ngram_range=(1,4))
bow = vec.fit_transform(df['task'])

CPU times: user 225 ms, sys: 1.55 ms, total: 227 ms
Wall time: 483 ms


In [217]:
query_list = np.array(vec.get_feature_names_out())
query_list

array(['aa', 'aa bb', 'aa bb cc', ..., 'ящика определите',
       'ящика определите по', 'ящика определите по данным'], dtype=object)

In [218]:
len(query_list)

34266

# Autocomplete

##BERT MASKING NEXT TOKEN

We don't use any large LLM for next token prediction for now, it's because...because they are large. If the project will use casual LLMs for some purposes, we can aplly the same LLM for searching engine.

In [ ]:
!pip install -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9

  Cloning https://github.com/huggingface/transformers (to revision de9255de27abfcae4a1f816b904915f0b1e23cd9) to /tmp/pip-req-build-ercx27jl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ercx27jl
  Running command git rev-parse -q --verify 'sha^de9255de27abfcae4a1f816b904915f0b1e23cd9'
  Running command git fetch -q https://github.com/huggingface/transformers de9255de27abfcae4a1f816b904915f0b1e23cd9
  Running command git checkout -q de9255de27abfcae4a1f816b904915f0b1e23cd9
  Resolved https://github.com/huggingface/transformers to commit de9255de27abfcae4a1f816b904915f0b1e23cd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/

In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

For getting embeddings and semantic search we apply bert-like model, so we can try to predict next token by masking it.

In [ ]:
# Define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
classifier = pipeline("fill-mask", model = "DeepPavlov/rubert-base-cased-sentence", tokenizer=tokenizer)
classifier("двухатомный идеальный [MASK]")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'score': 0.0002246604854008183,
  'token': 85588,
  'token_str': 'пени',
  'sequence': 'двухатомный идеальный пени'},
 {'score': 0.00018175203877035528,
  'token': 38132,
  'token_str': 'советы',
  'sequence': 'двухатомный идеальный советы'},
 {'score': 0.00016879251052159816,
  'token': 3813,
  'token_str': 'дан',
  'sequence': 'двухатомный идеальный дан'},
 {'score': 0.00015720840019639581,
  'token': 3111,
  'token_str': '勤',
  'sequence': 'двухатомный идеальный 勤'},
 {'score': 0.00015274326142389327,
  'token': 18349,
  'token_str': 'призван',
  'sequence': 'двухатомный идеальный призван'}]

### FAIL. But fine-tunung may help

##**TRIE**

###Var 1

In [219]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Autocomplete:
    def __init__(self, vocabulary):
        self.root = TrieNode()
        self.build_trie(vocabulary)

    def build_trie(self, vocabulary):
        for word in vocabulary:
            self.insert_word(word)

    def insert_word(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def find_words(self, prefix):
        node = self.root
        result = []
        for char in prefix:
            if char not in node.children:
                return result
            node = node.children[char]
        self.traverse_subtrie(node, prefix, result)
        return result

    def traverse_subtrie(self, node, prefix, result):
        if node.is_end_of_word:
            result.append(prefix)
        for char in node.children:
            self.traverse_subtrie(node.children[char], prefix + char, result)

In [220]:
vocabulary = query_list
autocomplete = Autocomplete(vocabulary)

In [224]:
%%time
prefix = 'площ'
results = autocomplete.find_words(prefix)
print(results[:5])

['площадей', 'площадей двух', 'площадей двух других', 'площадей других', 'площадей жёлтых']
CPU times: user 857 µs, sys: 0 ns, total: 857 µs
Wall time: 856 µs


In [226]:
%%time
prefix = 'кол'
results = autocomplete.find_words(prefix)
print(results[:5])

['колес', 'колес дышло', 'колес дышло будет', 'колес дышло будет всегда', 'колес почему']
CPU times: user 290 µs, sys: 0 ns, total: 290 µs
Wall time: 301 µs


In [229]:
%%time
prefix = 'те'
results = autocomplete.find_words(prefix)
print(results[:5])

['театра', 'театра чтобы', 'театра чтобы зрителям', 'театра чтобы зрителям из', 'тела']
CPU times: user 398 µs, sys: 0 ns, total: 398 µs
Wall time: 393 µs


In [230]:
%%time
prefix = 'кол'
results = autocomplete.find_words(prefix)
print(results[:5])

['колес', 'колес дышло', 'колес дышло будет', 'колес дышло будет всегда', 'колес почему']
CPU times: user 257 µs, sys: 22 µs, total: 279 µs
Wall time: 286 µs




---



###Var 2

In [231]:
class TrieNode:

    def __init__(self, c):
        self.data = c
        self.isEnd = False
        self.children = {} #map

class Trie:
    def __init__(self):
        self.root = TrieNode('')

    #Add a word to trie, Time O(s) Space O(1) s is word length
    def insert(self, word):
        node = self.root
        for char in word:
            if not char in node.children:
                node.children[char] = TrieNode(char)
            node = node.children[char]
        node.isEnd = True

  #find all word with given prefix
	#Time O(n), Space O(n), n is number of nodes involved (include prefix and branches)
    def autocomplete(self, word):
        res = []
        node = self.root
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                return []
        self.helper(node, res, word[:-1]) #except the last "ama", node is "z"
        return res

  #recursive function called by autocomplete
	#Time O(n), Space O(n), n is number of nodes in branches
    def helper(self, node, res, prefix):
        if node.isEnd:
            res.append(prefix + node.data)
        for child in node.children.values():
            self.helper(child, res, prefix + node.data)


In [232]:
t = Trie()
for query in query_list:
  t.insert(query)

In [233]:
%%time
a = t.autocomplete("яр")
print(a[:5])

['яркий', 'яркий солнечный', 'яркий солнечный день', 'яркий солнечный день на']
CPU times: user 145 µs, sys: 0 ns, total: 145 µs
Wall time: 151 µs


In [235]:
%%time
a = t.autocomplete("кол")
print(a[:5])

['колес', 'колес дышло', 'колес дышло будет', 'колес дышло будет всегда', 'колес почему']
CPU times: user 264 µs, sys: 1 µs, total: 265 µs
Wall time: 270 µs


In [236]:
%%time
a = t.autocomplete("колес ды")
print(a[:5])

['колес дышло', 'колес дышло будет', 'колес дышло будет всегда']
CPU times: user 486 µs, sys: 0 ns, total: 486 µs
Wall time: 457 µs


## **fast_autocomplete**

In [ ]:
!pip install fast-autocomplete[levenshtein]

In [ ]:
from fast_autocomplete import AutoComplete

In [ ]:
words = queries_dict
autocomplete = AutoComplete(words=words, valid_chars_for_string = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя') #add characters of rus alphabet
print(autocomplete.search(word='колеба', max_cost=5, size = 5))

[['поле'], ['d в поле'], ['0 выразив его'], ['n объем газа'], ['v б зная']]


FAIL.  not good with russian language